In [ ]:
## DERI KANSERI SINIFLADIRICI MODELI

# KÜTÜPHANELER
import os
import numpy as np
import tensorflow as tf
#from tensorflow import keras
#from keras.applications import imagenet_utils

from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense,Dropout, Flatten
#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import warnings
import glob
warnings.filterwarnings("ignore")

#Define Random seed
tf.random.set_seed(101)

In [ ]:
# Eğitim ve Doğrulama Görüntüleri için Yollar
data_dir = 'G:/My Drive/PYTHON WORKS/SKIN-LESION-CLASSIFIER/base_dir/'
train_path = data_dir + 'train_dir'
valid_path = data_dir + 'val_dir'

In [ ]:
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count = 0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count += len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [ ]:
# Girdi görüntüsünün yükseklik, genişlik ve renk ayarı.
IMG_CHANNELS = 3
IMG_ROWS = 224
IMG_COLS = 224
input_shape = (IMG_ROWS,IMG_COLS,3)

train_samples_num = get_files(train_path)
num_classes = len(glob.glob(train_path+"/*"))
val_samples_num   = get_files(valid_path)
#test_samples_num =  len(glob.glob(test_path+"/*"))

class_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel','nv', 'vasc']

# This function will plot images in the form of a grid with 1 row and 
# 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

# Birkaç yararlı bildirim
epochs = 15
num_train_samples = train_samples_num 
num_val_samples = val_samples_num
batch_size = 128
 
TARGET_SIZE = (IMG_ROWS, IMG_COLS)
OPTIM = tf.keras.optimizers.Adam(lr=0.001)
LOSS = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:

# Creating training and validation set
data_gen = ImageDataGenerator(rescale=1./255)

# Üreticileri ayarlama
train_batches = data_gen.flow_from_directory(train_path,
                                             target_size=TARGET_SIZE,
                                             batch_size=batch_size,
                                             color_mode='rgb',
                                             class_mode='sparse'
                                             )

valid_batches = data_gen.flow_from_directory(valid_path,
                                             target_size=TARGET_SIZE,
                                             batch_size=batch_size,
                                             color_mode='rgb',
                                             class_mode='sparse'
                                             )

test_batches = data_gen.flow_from_directory(valid_path,
                                            target_size=TARGET_SIZE,
                                            batch_size=batch_size,
                                            shuffle=False,
                                            color_mode='rgb',
                                            class_mode='sparse'
                                            )


classes = train_batches.class_indices # dictionary type data classes
# Take one image to visualize it's changes after every layer
augmented_images = [train_batches[0][0][0] for i in range(5)]
plotImages(augmented_images)

# Ön-eğitimli imagenet modeli oluşturma
#from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19
from keras.applications.densenet import preprocess_input
mobile = tf.keras.applications.VGG19(input_shape=input_shape,
                     weights='imagenet',
                     include_top=False) 

for layer in mobile.layers:
    layer.trainable = False
    
# imports the model and discards the last layer.
# Modeli düzenleme
x = mobile.output
x = GlobalAveragePooling2D()(x)
# layer to convert the features to a single element vector per image.
# let's add a fully-connected layer
x = Flatten()(x)
# Tahminler için dropout ve dense layer ekleme
x = Dropout(0.25)(x)
#x = Dense(512, activation='relu')(x) 
# we add dense layers so that the model can learn more complex 
# functions and classify for better results.
x = Dense(100, activation='relu')(x) #dense layer 2
#x = Dense(512,activation='relu')(x) #dense layer 3
# and a logistic layer -- let's say we have 7 classes
# final layer with softmax activation for 7 classes
predictions = Dense(num_classes, activation='softmax')(x)

# Yeni çıktılı modeli oluşturma
model = Model(inputs=mobile.input, outputs=predictions)

# örnek test görüntüsünü okuma
img_path = valid_path + '/bcc/ISIC_0024332.jpg'
img = image.load_img(img_path, target_size=TARGET_SIZE)
img = image.img_to_array(img)
img_test = img/255.
plt.imshow(img_test)
plt.show()

img_test = np.expand_dims(img_test, axis=0)
# getting model prediction for unseeen image
test_predictions = model.predict(img_test)
print(test_predictions)
predicted_label = np.argmax(test_predictions)

print(class_labels[predicted_label])

# let's visualize layer names and layer indices to see how many layers
for i, layer in enumerate(model.layers):
   print(i, layer.name)
# Modeldeki yeni katmanların özeti
model.summary()

In [ ]:
model_layers = [layer.name for layer in model.layers]
print('layer name : ', model_layers)

# Modeli derleme (should be done *after* setting layers to non-trainable)
model.compile(optimizer=OPTIM, loss=LOSS, metrics=['accuracy'])

# # Save the visualization as a file
# tf.keras.utils.plot_model(model, show_shapes=True, 
#                           to_file=data_dir + 'Transfer_CNN_diagram.png')

# Bir iterasyonda kaç adım gerektiğini bildirme
train_steps = train_batches.n // train_batches.batch_size
val_steps   = valid_batches.n // valid_batches.batch_size

# Modeli melanoma daha duyarlı hale getirmek için ağırlık ekleme
# class_weights={
#     0: 1.0,  # akiec
#     1: 1.0,  # bcc
#     2: 1.0,  # bkl
#     3: 1.0,  # df
#     4: 3.0,  # mel
#     5: 1.0,  # nv
#     6: 1.0,  # vasc
# }

# Save edilmiş model için dosya yolunu bildirme
filepath = data_dir + "model.h5"

# Modelin en iyi versiyonunu save etmek için checkpoint bildirme
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss', 
                             verbose=1,
                             save_best_only=True, 
                             mode='max')

# Öğrenme durgunlaştıkça öğrenme oranını azaltma
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.1, 
                              patience=2,
                              verbose=1, 
                              mode='max',
                              min_delta=1e-4,
                              min_lr=1e-4)

callbacks_list = [checkpoint, reduce_lr]

In [ ]:
# Modeli uydurma
history = model.fit(train_batches,
                    steps_per_epoch=train_steps,
                    validation_data=valid_batches,
                    validation_steps=val_steps,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks_list)

In [ ]:
# Eğitim sonuçlarını görselleştirme
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:

# En iyi dönüm için hesaplama
model.load_weights(filepath)

val_loss, val_acc = model.evaluate(test_batches, steps=val_steps)

print('val_loss:', val_loss)
print('val_cat_acc:', val_acc)

In [ ]:

# Test görüntülerinin confusion matrisini oluşturma
test_labels = test_batches.classes

# Tahmin yapma
predictions = model.predict(test_batches,
                            test_batches.n//test_batches.batch_size+1,
                            verbose=1)
predictions = np.argmax(predictions, axis=1)

# Confusion matrisi oluşturan fonksiyon
def plot_confusion_matrix(cm, classes, normalize=True, 
                          title='Confusion matris', cmap=plt.cm.Reds):
    """
    Bu fonksiyon, karışıklık matrisini yazdırır ve çizer. 
    'normalize=True' ile Normalleştirme uygulanabilir.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalize edilmiş karışıklık matrisi")
    else:
        print('Normalize edilmemiş Karışıklık matrisi')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Gerçek Etiket')
    plt.xlabel('Tahmini Etiket')
    plt.tight_layout()

cm = confusion_matrix(test_labels, predictions)

plot_confusion_matrix(cm, class_labels)

In [ ]:

# örnek test görüntüsünü okuma
img_path = valid_path + '/vasc/ISIC_0026876.jpg'
img = image.load_img(img_path, target_size=TARGET_SIZE)
img = image.img_to_array(img)
img_test = np.expand_dims(img/255., axis=0)
print("image shape: {}".format(img_test.shape))
plt.imshow(img_test)
plt.show()


img_test = preprocess_input(img_test)
# getting model predictions
test_predictions = model.predict(img_test)
print(test_predictions)
predicted_label = np.argmax(test_predictions)

print(class_labels[predicted_label])